# Testing fnGetFields
- Description: Based on pre-defined list schemas (MainListTest and LookupListTest), test the output of fnGetFields
- Version: v1
- Author: John Kerski

In [26]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from great_expectations.dataset import SparkDFDataset
import pandas as pd

#Load Data
delta_df = (spark.read.format("delta").load('Tables/Fields_To_Test'))
raw_test_df = SparkDFDataset(delta_df)
raw_test_pd_df = delta_df.toPandas()

# Display Delta
display(delta_df)


StatementMeta(, ec5dcd74-4a58-431c-83c9-45a9a5789166, 30, Finished, Available)

SynapseWidget(Synapse.DataFrame, 81d9f01a-7016-4fb7-b9d5-78340aee9432)

## Test the Schema of the Output of fnGetFields

In [27]:
MANDATORY_COLUMNS = [
    "Title",
    "Description",
    "Type_As_String",
    "Internal_Name",
    "Select_Parameter",
    "Expand_Parameter",
    "Table_Expand_Argument_-_Display_Name",
    "Table_Expand_Argument_-_Internal_Name",
    "List_Name"
]

for column in MANDATORY_COLUMNS:
    try:
        assert raw_test_df.expect_column_to_exist(column).success, f"FAILED: Mandatory column {column} does not exist."
        print(f"PASSED: Column {column} exists")
    except AssertionError as e:
        print(e)

StatementMeta(, ec5dcd74-4a58-431c-83c9-45a9a5789166, 31, Finished, Available)

PASSED: Column Title exists
PASSED: Column Description exists
PASSED: Column Type_As_String exists
PASSED: Column Internal_Name exists
PASSED: Column Select_Parameter exists
PASSED: Column Expand_Parameter exists
PASSED: Column Table_Expand_Argument_-_Display_Name exists
PASSED: Column Table_Expand_Argument_-_Internal_Name exists
PASSED: Column List_Name exists


## List_Name should be unique

In [28]:
list_name_count = len(pd.unique(raw_test_pd_df['List_Name']))
type(list_name_count)
try:
    assert list_name_count == 1, "FAILED: List_Name should have only 1 value."
    print(f"PASSED: List_Name contained only 1 value.")
except AssertionError as e:
    print(e)


StatementMeta(, ec5dcd74-4a58-431c-83c9-45a9a5789166, 32, Finished, Available)

PASSED: List_Name contained only 1 value.


## Based on upstream list, Type_As_String should have all the expected field types

In [29]:
MANDATORY_COLUMN_VALUES = [
"Boolean",
"Choice",
"Counter",
"Currency",
"DateTime",
"Location",
"Lookup",
"LookupMulti",
"MultiChoice",
"Note",
"Number",
"Text",
"Thumbnail",
"URL",
"User",
"UserMulti"
]

for column_value in MANDATORY_COLUMN_VALUES:
    try:
        assert (column_value in raw_test_pd_df['Type_As_String'].unique()) == True, "FAILED: Type_As_String has all the expected values: {column_value}"
        print(f"PASSED: Type_As_String has all the expected values: {column_value}")
    except AssertionError as e:
        print(e)




StatementMeta(, ec5dcd74-4a58-431c-83c9-45a9a5789166, 33, Finished, Available)

PASSED: Type_As_String has all the expected values: Boolean
PASSED: Type_As_String has all the expected values: Choice
PASSED: Type_As_String has all the expected values: Counter
PASSED: Type_As_String has all the expected values: Currency
PASSED: Type_As_String has all the expected values: DateTime
PASSED: Type_As_String has all the expected values: Location
PASSED: Type_As_String has all the expected values: Lookup
PASSED: Type_As_String has all the expected values: LookupMulti
PASSED: Type_As_String has all the expected values: MultiChoice
PASSED: Type_As_String has all the expected values: Note
PASSED: Type_As_String has all the expected values: Number
PASSED: Type_As_String has all the expected values: Text
PASSED: Type_As_String has all the expected values: Thumbnail
PASSED: Type_As_String has all the expected values: URL
PASSED: Type_As_String has all the expected values: User
PASSED: Type_As_String has all the expected values: UserMulti


## UserMulti has '/' in Select_Parameter and Expand_Parameter is not null

In [30]:
#Using variable
column_check='UserMulti'
user_multi_df=raw_test_pd_df[raw_test_pd_df["Type_As_String"] == column_check]

try:
    assert user_multi_df["Select_Parameter"].str.contains("/").bool() == True, "FAILED: Select_Parameter has '/' to indicated correct odata projection."
    print(f"PASSED: Select_Parameter has '/' to indicated correct odata projection.")
except AssertionError as e:
    print(e)

try:
    assert user_multi_df["Expand_Parameter"].isnull().values.any() == False, "FAILED: Expand Parameter should not be null"
    print(f"PASSED: Expand Parameter should not be null")
except AssertionError as e:
    print(e)


StatementMeta(, ec5dcd74-4a58-431c-83c9-45a9a5789166, 34, Finished, Available)

PASSED: Select_Parameter has '/' to indicated correct odata projection.
PASSED: Expand Parameter should not be null
